In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score, RandomizedSearchCV
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pylab as py
import scipy.stats as stats 
from scipy.stats import norm, skew, probplot
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [2]:
from Feat_eng import (Y_train, X_train, Y_test, X_test, 
    c1_train, c2_train, c3_train, c1_test, c2_test, c3_test, 
    Y_c1train, Y_c2train, Y_c3train, Y_c1test, Y_c2test ,Y_c3test)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().dro

In [3]:
# Randomized parameter grid search on all features

rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": np.arange(10, 20, 1), 
          "max_samples": [.7,.8,.9]}
rf_random = RandomizedSearchCV(rf, params, n_iter=25, cv=5, n_jobs=-1)

rf_random.fit(X_train,Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'max_samples': [0.7, 0.8, 0.9],
                                        'min_samples_leaf': [6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])})

In [4]:
# Run a PCA on all data:
scaler_pca = StandardScaler().fit(X_train)
features = scaler_pca.transform(X_train)
df1_scal = pd.DataFrame(features, columns = X_train.columns)

pca = PCA(n_components = 0.95)
df_reduced = pca.fit_transform(df1_scal)

df_reduced = pd.DataFrame(pca.components_, columns = X_train.columns, index = list(range(1,df_reduced.shape[1]+1)))
df_reduced = df_reduced.apply(lambda x: abs(x))

feat = []
for i in range(0,df_reduced.shape[0]):
    ff = df_reduced.iloc[i].idxmax()
    df_reduced.drop(ff, inplace = True, axis = 1)
    feat.append(ff)

feat.append('log_value')
X_PCA_train = X_train[feat]
X_PCA_test = X_test[feat]


In [5]:
# Randomized parameter grid search for PCA on all data:
rf2 = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf2_random = RandomizedSearchCV(rf2, params, n_iter=50, cv=5, n_jobs=-1)

rf2_random.fit(X_PCA_train,Y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [6]:
# Results of Random Forest on all features and data
print(rf_random.best_params_)
print(rf_random.best_score_)
Y_pred = rf_random.predict(X_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf_random.predict(X_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 19, 'min_samples_split': 8, 'min_samples_leaf': 7, 'max_samples': 0.9}
0.9789359852215291
The training RMSE is: 50.4575879615785
The test RMSE is: 75.36390298322046


In [7]:
# Create a dictionary of the results
r2_train = rf_random.best_score_
r2_test = rf_random.score(X_test, Y_test)
rf_output = pd.DataFrame()
rf_output['all'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all
RMSE_train,50.457588
RMSE_test,75.363903
R2_train,0.978936
R2_test,0.988983


In [8]:
# Results of Random Forest on all features with PCA
print(rf2_random.best_params_)
print(rf2_random.best_score_)
Y_pred = rf2_random.predict(X_PCA_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf2_random.predict(X_PCA_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 150, 'min_samples_split': 12, 'min_samples_leaf': 7, 'max_samples': 0.6}
0.979525803250505
The training RMSE is: 56.80838419033132
The test RMSE is: 73.56766046367797


In [9]:
# Create a dictionary of the all_PCA results
r2_train = rf2_random.best_score_
r2_test = rf2_random.score(X_PCA_test, Y_test)
rf_output['All_PCA'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA
RMSE_train,50.457588,56.808384
RMSE_test,75.363903,73.567660
R2_train,0.978936,0.979526
R2_test,0.988983,0.989525


In [10]:
# Create pickle files for both random forest models
import pickle
pickle.dump(rf_random, open('rf_all.pkl','wb'))
pickle.dump(rf2_random, open('rf_all_PCA.pkl','wb'))

## Cluster 1 Random Forest

In [11]:
# Randomized parameter grid search for Cluster 1:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf1= RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf1.fit(c1_train,Y_c1train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [12]:
# Results of Random Forest on Cluster 1:
print(rf1.best_params_)
print(rf1.best_score_)
Y_pred = rf1.predict(c1_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c1train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf1.predict(c1_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c1test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 150, 'min_samples_split': 8, 'min_samples_leaf': 5, 'max_samples': 0.8}
0.959260713623913
The training RMSE is: 61.090916689244445
The test RMSE is: 79.83897507338445


In [13]:
# Create a dictionary of the cluster 1 results
r2_train = rf1.best_score_
r2_test = rf1.score(c1_test, Y_c1test)
rf_output['Cluster_1'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA,Cluster_1
RMSE_train,50.457588,56.808384,61.090917
RMSE_test,75.363903,73.567660,79.838975
R2_train,0.978936,0.979526,0.959261
R2_test,0.988983,0.989525,0.985582


## Cluster 2 Random Forest

In [14]:
# Randomized parameter grid search for Cluster 2:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf2 = RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf2.fit(c2_train,Y_c2train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [15]:
# Results of Random Forest on Cluster 2:
print(rf2.best_params_)
print(rf2.best_score_)
Y_pred = rf2.predict(c2_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c2train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf2.predict(c2_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c2test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 150, 'min_samples_split': 12, 'min_samples_leaf': 5, 'max_samples': 0.5}
0.9760338070809083
The training RMSE is: 52.06657371283588
The test RMSE is: 71.92204130549759


In [16]:
# Create a dictionary of the cluster 2 results
r2_train = rf2.best_score_
r2_test = rf2.score(c2_test, Y_c2test)
rf_output['Cluster_2'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA,Cluster_1,Cluster_2
RMSE_train,50.457588,56.808384,61.090917,52.066574
RMSE_test,75.363903,73.567660,79.838975,71.922041
R2_train,0.978936,0.979526,0.959261,0.976034
R2_test,0.988983,0.989525,0.985582,0.986566


## Cluster 3 Random Forest

In [17]:
# Randomized parameter grid search for Cluster 3:
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf3 = RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf3.fit(c3_train,Y_c3train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [18]:
# Results of Random Forest on Cluster 3:
print(rf3.best_params_)
print(rf3.best_score_)
Y_pred = rf3.predict(c3_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c3train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf3.predict(c3_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c3test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 150, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_samples': 0.8}
0.9653572041591522
The training RMSE is: 43.78977584568195
The test RMSE is: 85.3710224090214


In [19]:
# Create a dictionary of the cluster 3 results
r2_train = rf3.best_score_
r2_test = rf3.score(c3_test, Y_c3test)
rf_output['Cluster_3'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA,Cluster_1,Cluster_2,Cluster_3
RMSE_train,50.457588,56.808384,61.090917,52.066574,43.789776
RMSE_test,75.363903,73.567660,79.838975,71.922041,85.371022
R2_train,0.978936,0.979526,0.959261,0.976034,0.965357
R2_test,0.988983,0.989525,0.985582,0.986566,0.984261


In [20]:
# Create pickle files for all clusters:
pickle.dump(rf1, open('rf_cluster1.pkl','wb'))
pickle.dump(rf2, open('rf_cluster2.pkl','wb'))
pickle.dump(rf3, open('rf_cluster3.pkl','wb'))

# Cluster with PCA
## Cluster 1 with PCA

In [22]:
# Randomized parameter grid search for Cluster 1 with PCA:
c1_PCA_train = c1_train[feat]
c1_PCA_test = c1_test[feat]
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf1_pca= RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf1_pca.fit(c1_PCA_train,Y_c1train)


/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [23]:
# Results of Random Forest on Cluster 1 with PCA:
print(rf1_pca.best_params_)
print(rf1_pca.best_score_)
Y_pred = rf1_pca.predict(c1_PCA_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c1train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf1_pca.predict(c1_PCA_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c1test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 6, 'max_samples': 0.8}
0.9600354465681218
The training RMSE is: 66.82546911538869
The test RMSE is: 78.55002499894074


In [24]:
# Create a dictionary of the cluster 1 results
r2_train = rf1_pca.best_score_
r2_test = rf1_pca.score(c1_PCA_test, Y_c1test)
rf_output['PCA_Cluster_1'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA,Cluster_1,Cluster_2,Cluster_3,PCA_Cluster_1
RMSE_train,50.457588,56.808384,61.090917,52.066574,43.789776,66.825469
RMSE_test,75.363903,73.567660,79.838975,71.922041,85.371022,78.550025
R2_train,0.978936,0.979526,0.959261,0.976034,0.965357,0.960035
R2_test,0.988983,0.989525,0.985582,0.986566,0.984261,0.986339


## Cluster 2 with PCA

In [25]:
# Randomized parameter grid search for Cluster 2 with PCA:
c2_PCA_train = c2_train[feat]
c2_PCA_test = c2_test[feat]
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf2_pca = RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf2_pca.fit(c2_PCA_train,Y_c2train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [26]:
# Results of Random Forest on Cluster 2:
print(rf2_pca.best_params_)
print(rf2_pca.best_score_)
Y_pred = rf2_pca.predict(c2_PCA_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c2train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf2_pca.predict(c2_PCA_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c2test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 5, 'max_samples': 0.8}
0.976508031867907
The training RMSE is: 45.83041879790641
The test RMSE is: 71.2190332362072


In [27]:
# Create a dictionary of the cluster 2 results
r2_train = rf2_pca.best_score_
r2_test = rf2_pca.score(c2_PCA_test, Y_c2test)
rf_output['PCA_Cluster_2'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA,Cluster_1,Cluster_2,Cluster_3,PCA_Cluster_1,PCA_Cluster_2
RMSE_train,50.457588,56.808384,61.090917,52.066574,43.789776,66.825469,45.830419
RMSE_test,75.363903,73.567660,79.838975,71.922041,85.371022,78.550025,71.219033
R2_train,0.978936,0.979526,0.959261,0.976034,0.965357,0.960035,0.976508
R2_test,0.988983,0.989525,0.985582,0.986566,0.984261,0.986339,0.986450


## Cluster 3 with PCA

In [28]:
# Randomized parameter grid search for Cluster 3:
c3_PCA_train = c3_train[feat]
c3_PCA_test = c3_test[feat]
rf = RandomForestRegressor(bootstrap = True, oob_score=True)
params = {"min_samples_leaf": [4,5,6,7,8], 
          "min_samples_split": np.linspace(start=2, stop=30, num=15, dtype=int), 
          "n_estimators": [50, 100, 150, 200], 
          "max_samples": [.4,.5,.6,.7,.8]}
rf3_pca = RandomizedSearchCV(rf, params, n_iter=50, cv=5, n_jobs=-1)

rf3_pca.fit(c3_PCA_train,Y_c3train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(oob_score=True),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'max_samples': [0.4, 0.5, 0.6, 0.7,
                                                        0.8],
                                        'min_samples_leaf': [4, 5, 6, 7, 8],
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                                        'n_estimators': [50, 100, 150, 200]})

In [29]:
# Results of Random Forest on Cluster 3:
print(rf3_pca.best_params_)
print(rf3_pca.best_score_)
Y_pred = rf3_pca.predict(c3_PCA_train)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c3train)
train_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The training RMSE is:', train_RMSE)
Y_pred = rf3_pca.predict(c3_PCA_test)
pred_unlog = np.exp(Y_pred)
Y_unlog = np.exp(Y_c3test)
test_RMSE = np.sqrt(mean_squared_error(Y_unlog, pred_unlog))
print('The test RMSE is:', test_RMSE)

{'n_estimators': 100, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_samples': 0.8}
0.9669781974385762
The training RMSE is: 41.26601321572755
The test RMSE is: 83.8169558771746


In [30]:
# Create a dictionary of the cluster 3 results
r2_train = rf3_pca.best_score_
r2_test = rf3_pca.score(c3_PCA_test, Y_c3test)
rf_output['PCA_Cluster_3'] = [train_RMSE, test_RMSE, r2_train, r2_test]
rf_output.index = ['RMSE_train', 'RMSE_test', 'R2_train', 'R2_test']
rf_output

,all,All_PCA,Cluster_1,Cluster_2,Cluster_3,PCA_Cluster_1,PCA_Cluster_2,PCA_Cluster_3
RMSE_train,50.457588,56.808384,61.090917,52.066574,43.789776,66.825469,45.830419,41.266013
RMSE_test,75.363903,73.567660,79.838975,71.922041,85.371022,78.550025,71.219033,83.816956
R2_train,0.978936,0.979526,0.959261,0.976034,0.965357,0.960035,0.976508,0.966978
R2_test,0.988983,0.989525,0.985582,0.986566,0.984261,0.986339,0.986450,0.984515


In [31]:
# Create pickle files for all clusters:
pickle.dump(rf1_pca, open('rf1_pca.pkl','wb'))
pickle.dump(rf2_pca, open('rf2_pca.pkl','wb'))
pickle.dump(rf3_pca, open('rf3_pca.pkl','wb'))